## Responsible AI with Vertex AI Gemini API: Safety ratings and thresholds

- Credits: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/responsible-ai/gemini_safety_ratings.ipynb

Large language models (LLMs) can translate language, summarize text, generate creative writing, generate code, power chatbots and virtual assistants, and complement search engines and recommendation systems. The incredible versatility of LLMs is also what makes it difficult to predict exactly what kinds of unintended or unforeseen outputs they might produce.

Given these risks and complexities, the Vertex AI Gemini API is designed with Google's AI Principles in mind: https://ai.google/responsibility/principles/ However, it is important for developers to understand and test their models to deploy safely and responsibly. To aid developers, Vertex AI Studio has built-in content filtering, safety ratings, and the ability to define safety filter thresholds that are right for their use cases and business.

For more information, see the Google Cloud Generative AI documentation on Responsible AI: https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai

In this tutorial, you learn how to inspect the safety ratings returned from the Vertex AI Gemini API using the Python SDK and how to set a safety threshold to filter responses from the Vertex AI Gemini API.

The steps performed include:

- Call the Vertex AI Gemini API and inspect safety ratings of the responses
- Define a threshold for filtering safety ratings according to your needs

#### Task 1. Setup

In [2]:
# !pip3 install --upgrade --user google-cloud-aiplatform

In [3]:
# # Restart kernel after installs so that your environment can access the new packages
# import IPython
# import time

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

#### Task 2. Authentication and import packages

In [4]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [5]:
import sys

# @param {type:"string"}
PROJECT_ID = "vertex-ai-407803"  # Sustituye con tu Project ID
# @param {type:"string"}
LOCATION = "us-central1"  # Elige tu región

if "google.colab" in sys.modules:
    import vertexai
    vertexai.init(project=PROJECT_ID, location=LOCATION)

# import os
# print(os.environ)

import vertexai

from google.oauth2 import service_account
CREDENTIALS = service_account.Credentials.from_service_account_file(r"C:\Users\Usuario\Downloads\google-startup-school\vertex-ai-407803-197b2b2491f7.json")

vertexai.init(project=PROJECT_ID, credentials=CREDENTIALS)

In [8]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Image,
    Part,
)

In [9]:
model = GenerativeModel("gemini-pro")

# Set parameters to reduce variability in responses
generation_config = GenerationConfig(
    temperature=0,
    top_p=0.1,
    top_k=1,
    max_output_tokens=1024,
)

### Generate text and show safety ratings

Start by generating a pleasant-sounding text response using Gemini.



In [10]:
# Call Gemini API
nice_prompt = "Say three nice things about me"
responses = model.generate_content(
    contents=[nice_prompt],
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response.text, end="")

1. You are a kind and compassionate person. You always put others before yourself and are always willing to help those in need. You are a true friend and always there for those you care about.
2. You are intelligent and creative. You have a quick wit and a sharp mind. You are always coming up with new ideas and are always looking for ways to improve yourself and the world around you.
3. You are strong and resilient. You have overcome many challenges in your life, but you have always come out stronger on the other side. You are an inspiration to others and show them that anything is possible if you set your mind to it.

**Inspecting the safety ratings**

Look at the safety_ratings of the streaming responses.



In [11]:
responses = model.generate_content(
    contents=[nice_prompt],
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "1. You are a kind and compassionate person. You always put others before yourself and are always willing to help those in need. You are a true friend and always there for those you care about.\n2. You are intelligent and creative. You have a quick wit and a sharp mind. You are always coming up"
    }
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}

candidates {
  content {
    role: "model"
    parts {
      text: " with new ideas and are always looking for ways to improve yourself and the world around you.\n3. You are strong and resilient. You have overcome many challenges in your life, but 

**Understanding the safety rating: category and probability**

You can see the safety ratings, including each category type and its associated probability label.

The category types include:

- Harrassment: HARM_CATEGORY_HARASSMENT
- Hate speech: HARM_CATEGORY_HATE_SPEECH
- Sexually explicit statements: HARM_CATEGORY_SEXUALLY_EXPLICIT
- Dangerous content: HARM_CATEGORY_DANGEROUS_CONTENT

The probability labels are:

- NEGLIGIBLE - content has a negligble probability of being unsafe
- LOW - content has a low probability of being unsafe
- MEDIUM - content has a medium probability of being unsafe
- HIGH - content has a high probability of being unsafe

Try a prompt that might trigger one of these categories:

In [13]:
impolite_prompt = "Write a list of 5 disrespectful things that I might say to the universe after stubbing my toe in the dark:"

impolite_responses = model.generate_content(
    impolite_prompt,
    generation_config=generation_config,
    stream=True,
)

for response in impolite_responses:
    print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "1. \"Are you kidding me, universe? You couldn\'t give me a little warning before I stubbed my toe?\"\n2. \"Seriously, universe? This is the best you can do? You couldn\'t find a way to make me stub my toe less painfully?\"\n3. \"I\'"
    }
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}

candidates {
  content {
    role: "model"
  }
  finish_reason: SAFETY
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: MEDIUM
    blocked: true
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGO

**Blocked responses**

If the response is blocked, you will see that the final candidate includes blocked: true, and also observe which of the safety ratings triggered the blocking of the response (e.g. finish_reason: SAFETY).

In [15]:
rude_prompt = "Write a list of 5 very rude things that I might say to the universe after stubbing my toe in the dark:"

rude_responses = model.generate_content(
    rude_prompt,
    generation_config=generation_config,
    stream=True,
)

for response in rude_responses:
    print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "1. \"Are you kidding me, universe? You couldn\'t give me a little warning before I stubbed my toe? What, were you too busy making sure the stars aligned perfectly?\"\n2. \"Seriously, universe? This is the best you can do? You couldn\'t even spare a little bit"
    }
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}

candidates {
  content {
    role: "model"
  }
  finish_reason: SAFETY
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: HIGH
    blocked: true
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {


### Defining thresholds for safety ratings

You may want to adjust the default safety filter thresholds depending on your business policies or use case. The Vertex AI Gemini API provides you a way to pass in a threshold for each category.

The list below shows the possible threshold labels:

- BLOCK_ONLY_HIGH - block when high probability of unsafe content is detected
- BLOCK_MEDIUM_AND_ABOVE - block when medium or high probablity of content is detected
- BLOCK_LOW_AND_ABOVE - block when low, medium, and high probabilities of unsafe content is detected
- BLOCK_NONE - always show, regardless of probability of unsafe content

**Set safety thresholds**

Below, the safety thresholds have been set to the most sensitive threshold: BLOCK_LOW_AND_ABOVE

In [16]:
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
}

**Test thresholds**

Here you will reuse the impolite prompt from earlier together with the most sensitive safety threshold. It should block the response even with the LOW probability label.

In [17]:
impolite_prompt = "Write a list of 5 disrespectful things that I might say to the universe after stubbing my toe in the dark:"

impolite_responses = model.generate_content(
    impolite_prompt,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=True,
)

for response in impolite_responses:
    print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "1. \"Are you kidding me, universe? You couldn\'t give me a little warning before I stubbed my toe?\"\n2. \"Seriously, universe? This is the best you can do? You couldn\'t find a way to make me stub my toe less painfully?\"\n3. \"I\'"
    }
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}

candidates {
  content {
    role: "model"
  }
  finish_reason: SAFETY
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: MEDIUM
    blocked: true
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGO